In [2]:
import torch
import ase.io
from amptorch.trainer import AtomsTrainer
import amptorch

from pathlib import Path

In [14]:
amptorch_path = Path("../../amptorch")
train_data_path = Path("../oc20_3k_train.traj")

In [15]:
train_data = ase.io.read(train_data_path)

elements = list(set(atom.symbol for atom in train_data))

def get_path_to_gaussian(element):
    gaussians_path = amptorch_path / "examples/GMP/valence_gaussians"
    return next(p for p in gaussians_path.iterdir() if p.name.startswith(element + "_"))

atom_gaussians = {element: get_path_to_gaussian(element) for element in elements}

sigmas = [0.2, 0.69, 1.1, 1.66, 2.66]

MCSHs = {
    "MCSHs": {
        "0": {"groups": [1], "sigmas": sigmas},
        "1": {"groups": [1], "sigmas": sigmas},
        "2": {"groups": [1, 2], "sigmas": sigmas},
    },
    "atom_gaussians": atom_gaussians,
    "cutoff": 8,
}


In [16]:
dataset_config = {
    "raw_data": train_data_path,
    "fp_scheme": "gmp",
    "fp_params": MCSHs,
    "elements": elements,
    "save_fps": True,
    "scaling": {"type": "normalize", "range": (0, 1)},
    "val_split": 0.1,
}

cmd_config = 

In [19]:
def calc_loss(**params):
    model_params = ["get_forces", "num_layers", "num_nodes", "batchnorm"]
    optim_params = ["force_coefficient", "lr", "batch_size", "epochs", "loss", "metric"]
    
    config = {
        "model": {"name": "singlenn"},
        "optim": {},
        "dataset": {
            "raw_data": str(train_data_path),
            "fp_scheme": "gmp",
            "fp_params": MCSHs,
            "elements": elements,
            "save_fps": True,
            "scaling": {"type": "normalize", "range": (0, 1)},
            "val_split": 0.1,
        },
        "cmd": {
            "debug": False,
            "run_dir": "./",
            "seed": 1,
            "identifier": "test",
            "verbose": True,
            "logger": False,
        },
    }
    
    for k in model_params:
        config["model"][k] = params[k]
    for k in optim_params:
        config["optim"][k] = params[k]
        
    trainer = AtomsTrainer(config)
    trainer.train()

In [20]:
calc_loss(
    get_forces=True,
    num_layers=3,
    num_nodes=10,
    batchnorm=True,
    force_coefficient=0,
    lr=1e-3,
    batch_size=16,
    epochs=10,
    loss="mse",
    metric="mae",
)

Results saved to ./checkpoints/2022-02-14-12-58-31-test



Loading dataset: 3000 images
Use Xavier initialization
Loading model: 501 parameters
Loading skorch trainer
  epoch    train_energy_mae    train_forces_mae    train_loss    val_energy_mae    val_forces_mae    valid_loss    cp     dur
-------  ------------------  ------------------  ------------  ----------------  ----------------  ------------  ----  ------
      1           1066.3795             55.6408       33.2362          501.5571           34.6613        7.4407     +  6.4808


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


      2            441.6300             32.3773        4.8012          327.7580           26.6124        2.3287     +  6.4146
      3            338.0735             22.7018        2.5142          438.6731           20.8129        3.1687        6.4898
      4            295.9209             16.5670        1.9177          279.4230           17.1901        1.4291     +  6.4499
      5            253.7717             13.4289        1.3217          398.5257           15.6561        2.3555        7.1460
      6            248.9843             11.5695        1.1249          239.2520           12.3963        1.0510     +  6.5255
      7            233.0127              9.5620        0.9323          238.6214           10.7500        1.1394     +  6.4103
      8            215.1974              8.2029        0.7460          215.6015           10.0887        0.9749     +  6.5280
      9            215.0403              7.8546        0.8876          202.5410            9.1267        0.6670     + 